# **3. Preprocesamiento de Datos**


In [14]:
# === Sección 3: Preprocesamiento ===
from pathlib import Path
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Rutas del proyecto
ROOT = Path("..")
DATA = ROOT / "data"
RAW = DATA / "raw"
PROC = DATA / "processed"
MODELS = ROOT / "models"
for p in [PROC, MODELS]:
    p.mkdir(parents=True, exist_ok=True)


In [15]:
# Usamos el parquet que guardaste en el EDA
PARQUET = PROC / "telco_churn.parquet"
CSV = RAW / "telco_churn.csv"
df = pd.read_parquet(PARQUET) if PARQUET.exists() else pd.read_csv(CSV)

# Normalización mínima de nombres (por si no lo hiciste antes)
df.columns = (
    df.columns
      .str.strip()
      .str.replace(" ", "_")
      .str.replace("(", "", regex=False)
      .str.replace(")", "", regex=False)
)

# Tipos clave
TARGET = "Churn"
ID_COL = "customerID" if "customerID" in df.columns else "customerid"

# Asegurar tipos esperados
if df[TARGET].dtype == "O":
    df[TARGET] = df[TARGET].map({"Yes":1, "No":0}).astype("int8")

# TotalCharges a numérico (en este dataset puede venir sucio)
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")


In [16]:
# Detectores automáticos
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
num_cols = df.select_dtypes(include=[np.number, "bool"]).columns.tolist()

# Quitar columnas que no deben transformarse como categóricas
if ID_COL in cat_cols: cat_cols.remove(ID_COL)
if TARGET in cat_cols: cat_cols.remove(TARGET)
if TARGET in num_cols: num_cols.remove(TARGET)

print("Numéricas:", len(num_cols), num_cols)
print("Categóricas:", len(cat_cols), cat_cols)



Numéricas: 4 ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']
Categóricas: 15 ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [17]:

X = df.drop(columns=[TARGET])
y = df[TARGET].astype("int8")

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, y_train.mean().round(3))
print("Test: ", X_test.shape, y_test.mean().round(3))


Train: (5634, 20) 0.265
Test:  (1409, 20) 0.265


In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class ManejoAtipicos(BaseEstimator, TransformerMixin):
    def __init__(self, metodo="percentil", estrategia="winsorizar",
                 columns=None, lower=0.01, upper=0.99):
        self.metodo = metodo
        self.estrategia = estrategia   # 'winsorizar' recomendado en pipeline
        self.columns = columns         # lista de nombres de columnas numéricas
        self.lower = float(lower)
        self.upper = float(upper)
        self.limites_ = {}
        self.feature_names_in_ = None
        self.dtypes_ = None
        self.transform_cols_ = None
        self.last_mask_ = None

    # --- helpers ---
    def _to_frame(self, X):
        """Devuelve un DataFrame con nombres de columnas consistentes."""
        if isinstance(X, pd.DataFrame):
            return X.copy()
        X = np.asarray(X)
        if self.columns is not None:
            cols = list(self.columns)
        elif self.feature_names_in_ is not None:
            cols = list(self.feature_names_in_)
        else:
            cols = [f"x{i}" for i in range(X.shape[1])]
        return pd.DataFrame(X, columns=cols).copy()

    def fit(self, X, y=None):
        X = self._to_frame(X)
        # Si no especificaste columns, usa todas las numéricas
        cols = list(self.columns) if self.columns is not None else \
               X.select_dtypes(include=[np.number]).columns.tolist()

        self.feature_names_in_ = list(X.columns)
        self.transform_cols_ = cols
        self.dtypes_ = X.dtypes.to_dict()

        self.limites_.clear()
        for col in cols:
            s = pd.to_numeric(X[col], errors="coerce")
            if self.metodo == "iqr":
                q1, q3 = s.quantile(0.25), s.quantile(0.75)
                iqr = q3 - q1
                li, ls = q1 - 1.5*iqr, q3 + 1.5*iqr
            elif self.metodo == "percentil":
                li, ls = s.quantile(self.lower), s.quantile(self.upper)
            else:
                raise ValueError("Método no soportado")
            self.limites_[col] = (li, ls)
        return self

    def row_mask(self, X):
        """Máscara booleana (True = fila dentro de límites). Útil si deseas ELIMINAR fuera del pipeline."""
        X = self._to_frame(X)
        mask = np.ones(len(X), dtype=bool)
        for col in self.transform_cols_:
            s = pd.to_numeric(X[col], errors="coerce")
            li, ls = self.limites_[col]
            mask &= (s >= li) & (s <= ls)
        return mask

    def transform(self, X):
        X = self._to_frame(X)
        cols = [c for c in (self.transform_cols_ or []) if c in X.columns]

        if self.estrategia == "winsorizar":
            for col in cols:
                s = pd.to_numeric(X[col], errors="coerce")
                li, ls = self.limites_[col]
                X[col] = np.clip(s, li, ls)
            # Restaurar dtypes donde aplique
            for c in X.columns:
                try:
                    X[c] = X[c].astype(self.dtypes_.get(c, X[c].dtype))
                except Exception:
                    pass
            return X.values  # devolver ndarray para jugar bien con ColumnTransformer

        elif self.estrategia == "eliminar":
            # ⚠️ No usar dentro de un Pipeline (cambia n_muestras).
            mask = self.row_mask(X)
            self.last_mask_ = mask
            X = X.loc[mask].reset_index(drop=True)
            for c in X.columns:
                try:
                    X[c] = X[c].astype(self.dtypes_.get(c, X[c].dtype))
                except Exception:
                    pass
            return X.values

        else:
            raise ValueError("Estrategia no soportada")

    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names_in_ if input_features is None else input_features)



In [19]:
scale_numeric = False  # <- cambia a True si vas a usar modelos que lo requieran

numeric_steps = [
    ("imputer", SimpleImputer(strategy="median")),
]
if scale_numeric:
    numeric_steps.append(("scaler", StandardScaler()))

numeric_transformer = Pipeline(steps=numeric_steps) if scale_numeric else SimpleImputer(strategy="median")

categorical_transformer = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

# ColumnTransformer que aplica lo anterior por tipo de columna
preprocessor = ColumnTransformer(
    transformers=[
        ("num",  numeric_transformer, num_cols),
        ("cat",  categorical_transformer, cat_cols)
    ],
    remainder="drop",
    verbose_feature_names_out=False
)


In [20]:
# Ajustar SOLO con train
preprocessor.fit(X_train)

# Transformar
X_train_t = preprocessor.transform(X_train)
X_test_t  = preprocessor.transform(X_test)

# Recuperar nombres de features tras OHE
feature_names = preprocessor.get_feature_names_out()
X_train_df = pd.DataFrame(X_train_t, columns=feature_names, index=X_train.index)
X_test_df  = pd.DataFrame(X_test_t,  columns=feature_names, index=X_test.index)

print("Shapes procesados:")
print("X_train:", X_train_df.shape, "| X_test:", X_test_df.shape)



Shapes procesados:
X_train: (5634, 45) | X_test: (1409, 45)


In [21]:
# Guardar datasets procesados
X_train_df.to_parquet(PROC/"X_train.parquet", index=False)
X_test_df.to_parquet(PROC/"X_test.parquet", index=False)
y_train.to_frame(name=TARGET).to_parquet(PROC/"y_train.parquet", index=False)
y_test.to_frame(name=TARGET).to_parquet(PROC/"y_test.parquet", index=False)

print("Guardados:")
print(" -", (PROC/"X_train.parquet").resolve())
print(" -", (PROC/"X_test.parquet").resolve())
print(" -", (PROC/"y_train.parquet").resolve())
print(" -", (PROC/"y_test.parquet").resolve())

# Guardar el preprocesador (útil para GridSearchCV y FastAPI)
joblib.dump(preprocessor, MODELS/"preprocessor.joblib")
print("Preprocessor guardado en:", (MODELS/"preprocessor.joblib").resolve())



Guardados:
 - C:\Users\juana\MLOPS\miniproyecto6\data\processed\X_train.parquet
 - C:\Users\juana\MLOPS\miniproyecto6\data\processed\X_test.parquet
 - C:\Users\juana\MLOPS\miniproyecto6\data\processed\y_train.parquet
 - C:\Users\juana\MLOPS\miniproyecto6\data\processed\y_test.parquet
Preprocessor guardado en: C:\Users\juana\MLOPS\miniproyecto6\models\preprocessor.joblib


In [22]:
X_train_df.isna().sum().sum(), X_test_df.isna().sum().sum()


(np.int64(0), np.int64(0))